In [7]:
print("hello world")
#hello can you see this new change
#please write your name just to make sure we can all edit this
# dilara

hello world


In [8]:
#Imports
import numpy as np
import scipy as sci

In [12]:
"""
Class for quantum register for n number of qubits
"""
class Quantum_Register:
    def __init__(self, n):
        self.n = n 
        self.psi = np.zeros((2,)*n)

In [17]:
"""Testing block """

newRegister= Quantum_Register(1)
print(newRegister.n)
print(newRegister.psi)

1
[0. 0.]
